# Analisando listagens de carros usados no eBay Kleinanzeigen


Estaremos trabalhando em um conjunto de dados de carros usados do eBay Kleinanzeigen, uma seção de classificados do site alemão eBay.

O conjunto de dados foi originalmente copiado e carregado no Kaggle. A versão do conjunto de dados com a qual estamos trabalhando é uma amostra de 50.000 pontos de dados preparados pelo Dataquest, incluindo a simulação de uma versão menos limpa dos dados.


O objetivo deste projeto é limpar os dados e analisar as listas de carros usados incluídos.

In [10]:
import pandas as pd
import numpy as np

In [11]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   dateCrawled          371528 non-null  object
 1   name                 371528 non-null  object
 2   seller               371528 non-null  object
 3   offerType            371528 non-null  object
 4   price                371528 non-null  int64 
 5   abtest               371528 non-null  object
 6   vehicleType          333659 non-null  object
 7   yearOfRegistration   371528 non-null  int64 
 8   gearbox              351319 non-null  object
 9   powerPS              371528 non-null  int64 
 10  model                351044 non-null  object
 11  kilometer            371528 non-null  int64 
 12  monthOfRegistration  371528 non-null  int64 
 13  fuelType             338142 non-null  object
 14  brand                371528 non-null  object
 15  notRepairedDamage    299468 non-nu

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


## Limpando as colunas

In [12]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

Faremos algumas alterações aqui:

- Altere as colunas de camelcase para snakecase.
- Altere algumas palavras para descrever as colunas com mais precisão.

In [13]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen']
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


## Exploração e limpeza inicial de dados
Começaremos explorando os dados para encontrar áreas óbvias onde podemos limpar os dados.

In [14]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,371528,371528,371528,371528,3.715280e+05,371528,333659,371528.000000,351319,371528.000000,351044,371528.000000,371528.000000,338142,371528,299468,371528,371528.0,371528.00000,371528
unique,280500,233531,2,2,NaN,2,8,NaN,2,NaN,251,NaN,NaN,7,40,2,114,NaN,NaN,182806
top,2016-03-24 14:49:47,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:45:59
freq,7,657,371525,371516,NaN,192585,95894,NaN,274214,NaN,30070,NaN,NaN,223857,79640,263182,14450,NaN,NaN,17
mean,NaN,NaN,NaN,NaN,1.729514e+04,NaN,NaN,2004.577997,NaN,115.549477,NaN,125618.688228,5.734445,NaN,NaN,NaN,NaN,0.0,50820.66764,NaN
std,NaN,NaN,NaN,NaN,3.587954e+06,NaN,NaN,92.866598,NaN,192.139578,NaN,40112.337051,3.712412,NaN,NaN,NaN,NaN,0.0,25799.08247,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.00000,NaN
25%,NaN,NaN,NaN,NaN,1.150000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30459.00000,NaN
50%,NaN,NaN,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49610.00000,NaN
75%,NaN,NaN,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71546.00000,NaN


 Nossas observações iniciais:

Há várias colunas de texto em que todos (ou quase todos) os valores são os mesmos:
- `saller`
- `offer_type`
A coluna `num_photos` parece estranha, precisaremos investigar isso mais a fundo.

In [15]:
autos["num_photos"].value_counts()

0    371528
Name: num_photos, dtype: int64

Parece que a coluna `num_photos` tem 0 para cada coluna. Iremos descartar esta coluna, mais as outras duas que anotamos como um único valor.

In [16]:
autos = autos.drop(["num_photos", "seller", "offer_type"], axis=1)

Existem duas colunas, `price` e `auto`, que são valores numéricos com caracteres extras sendo armazenados como texto. Vamos limpá-los e convertê-los.

In [27]:
autos["price"] = (autos["price"].astype(str).str.replace("$","").str.replace(",","").astype(int))
autos["price"].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


0      480
1    18300
2     9800
3     1500
4     3600
Name: price, dtype: int64

In [28]:
autos["odometer"] = (autos["odometer"].astype(str)
                             .str.replace("km","")
                             .str.replace(",","")
                             .astype(int)
                             )
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)
autos["odometer_km"].head()

0    150000
1    125000
2    125000
3    150000
4     90000
Name: odometer_km, dtype: int64

## Explorando odômetro e preço

In [29]:
autos["odometer_km"].value_counts()

150000    240797
125000     38067
100000     15920
90000      12523
80000      11053
70000       9773
60000       8669
50000       7615
5000        7069
40000       6376
30000       6041
20000       5676
10000       1949
Name: odometer_km, dtype: int64

Podemos observar que os valores neste campo são arredondados, o que pode indicar que os vendedores tiveram que escolher entre opções predefinidas para este campo. Além disso, há mais veículos com alta quilometragem do que com baixa quilometragem.

In [30]:
print(autos["price"].unique().shape)
print(autos["price"].describe())
autos["price"].value_counts().head(20)

(5597,)
count    3.715280e+05
mean     1.729514e+04
std      3.587954e+06
min      0.000000e+00
25%      1.150000e+03
50%      2.950000e+03
75%      7.200000e+03
max      2.147484e+09
Name: price, dtype: float64


0       10778
500      5670
1500     5394
1000     4649
1200     4594
2500     4438
600      3819
3500     3792
800      3784
2000     3432
999      3364
750      3203
650      3150
4500     3053
850      2946
2200     2936
700      2936
1800     2886
900      2874
950      2793
Name: price, dtype: int64

Novamente, os preços nesta coluna parecem arredondados, no entanto, como há 2.357 valores únicos na coluna, isso pode ser apenas a tendência das pessoas de arredondar os preços no site.

Há 1.421 carros listados com preço de US$ 0 - considerando que isso representa apenas 2% dos carros, podemos considerar a remoção dessas linhas. O preço máximo é de cem milhões de dólares, o que parece muito, vamos dar uma olhada nos preços mais altos.

In [31]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

2147483647     1
99999999      15
99000000       1
74185296       1
32545461       1
27322222       1
14000500       1
12345678       9
11111111      10
10010011       1
10000000       8
9999999        3
3895000        1
3890000        1
2995000        1
2795000        1
1600000        2
1300000        1
1250000        2
1234566        1
Name: price, dtype: int64

In [32]:
autos["price"].value_counts().sort_index(ascending=True).head(20)

0     10778
1      1189
2        12
3         8
4         1
5        26
7         3
8         9
9         8
10       84
11        5
12        8
13        7
14        5
15       27
16        2
17        5
18        3
19        3
20       51
Name: price, dtype: int64

In [33]:
autos = autos[autos["price"].between(1,351000)]
autos["price"].describe()

count    360635.000000
mean       5898.671956
std        8866.359669
min           1.000000
25%        1250.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

## Explorando as colunas de data
Há uma série de colunas com informações de data:

`date_crawled`

`registration_month`

`registration_year`

`ad_created`

`last_seen`


Estas são uma combinação de datas que foram rastreadas e datas com meta-informações do rastreador. As datas de não registro são armazenadas como strings.

Exploraremos cada uma dessas colunas para saber mais sobre as listagens.

In [34]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-24 11:52:17,2016-03-24 00:00:00,2016-04-07 03:16:57
1,2016-03-24 10:58:45,2016-03-24 00:00:00,2016-04-07 01:46:50
2,2016-03-14 12:52:21,2016-03-14 00:00:00,2016-04-05 12:47:46
3,2016-03-17 16:54:04,2016-03-17 00:00:00,2016-03-17 17:40:17
4,2016-03-31 17:25:20,2016-03-31 00:00:00,2016-04-06 10:17:21


In [35]:
(autos["date_crawled"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

2016-03-05    0.025547
2016-03-06    0.014483
2016-03-07    0.035657
2016-03-08    0.033469
2016-03-09    0.034115
2016-03-10    0.032645
2016-03-11    0.032773
2016-03-12    0.036242
2016-03-13    0.015783
2016-03-14    0.036330
2016-03-15    0.033424
2016-03-16    0.030205
2016-03-17    0.031647
2016-03-18    0.013119
2016-03-19    0.035271
2016-03-20    0.036400
2016-03-21    0.035682
2016-03-22    0.032493
2016-03-23    0.032002
2016-03-24    0.029914
2016-03-25    0.032800
2016-03-26    0.031974
2016-03-27    0.030227
2016-03-28    0.035063
2016-03-29    0.034126
2016-03-30    0.033535
2016-03-31    0.031872
2016-04-01    0.034145
2016-04-02    0.035094
2016-04-03    0.038812
2016-04-04    0.037628
2016-04-05    0.012780
2016-04-06    0.003128
2016-04-07    0.001617
Name: date_crawled, dtype: float64

In [36]:
(autos["date_crawled"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_values()
        )

2016-04-07    0.001617
2016-04-06    0.003128
2016-04-05    0.012780
2016-03-18    0.013119
2016-03-06    0.014483
2016-03-13    0.015783
2016-03-05    0.025547
2016-03-24    0.029914
2016-03-16    0.030205
2016-03-27    0.030227
2016-03-17    0.031647
2016-03-31    0.031872
2016-03-26    0.031974
2016-03-23    0.032002
2016-03-22    0.032493
2016-03-10    0.032645
2016-03-11    0.032773
2016-03-25    0.032800
2016-03-15    0.033424
2016-03-08    0.033469
2016-03-30    0.033535
2016-03-09    0.034115
2016-03-29    0.034126
2016-04-01    0.034145
2016-03-28    0.035063
2016-04-02    0.035094
2016-03-19    0.035271
2016-03-07    0.035657
2016-03-21    0.035682
2016-03-12    0.036242
2016-03-14    0.036330
2016-03-20    0.036400
2016-04-04    0.037628
2016-04-03    0.038812
Name: date_crawled, dtype: float64

Parece que o site foi rastreado diariamente durante aproximadamente um mês em março e abril de 2016. A distribuição das listagens rastreadas a cada dia é aproximadamente uniforme.

In [37]:
(autos["last_seen"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

2016-03-05    0.001264
2016-03-06    0.004098
2016-03-07    0.005202
2016-03-08    0.007939
2016-03-09    0.009824
2016-03-10    0.011460
2016-03-11    0.012955
2016-03-12    0.023240
2016-03-13    0.008410
2016-03-14    0.012176
2016-03-15    0.016324
2016-03-16    0.016418
2016-03-17    0.028699
2016-03-18    0.006888
2016-03-19    0.016330
2016-03-20    0.019884
2016-03-21    0.020026
2016-03-22    0.020508
2016-03-23    0.018015
2016-03-24    0.019163
2016-03-25    0.019000
2016-03-26    0.015958
2016-03-27    0.016721
2016-03-28    0.022189
2016-03-29    0.023284
2016-03-30    0.023725
2016-03-31    0.024243
2016-04-01    0.023897
2016-04-02    0.024967
2016-04-03    0.025308
2016-04-04    0.025536
2016-04-05    0.126962
2016-04-06    0.218950
2016-04-07    0.130437
Name: last_seen, dtype: float64

O rastreador registrou a data em que viu qualquer listagem pela última vez, o que nos permite determinar em que dia uma listagem foi removida, presumivelmente porque o carro foi vendido.

Os últimos três dias contêm uma quantidade desproporcional de valores 'vistos pela última vez'. Dado que esses valores são de 6 a 10 vezes os valores dos dias anteriores, é improvável que tenha havido um grande aumento nas vendas e, mais provavelmente, esses valores estão relacionados ao final do período de rastreamento e não indicam vendas de carros.

In [38]:
print(autos["ad_created"].str[:10].unique().shape)
(autos["ad_created"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

(114,)


2014-03-10    0.000003
2015-03-20    0.000003
2015-06-11    0.000003
2015-06-18    0.000003
2015-08-07    0.000003
                ...   
2016-04-03    0.039001
2016-04-04    0.037736
2016-04-05    0.011613
2016-04-06    0.003119
2016-04-07    0.001553
Name: ad_created, Length: 114, dtype: float64

Há uma grande variedade de datas de criação de anúncios. A maioria cai dentro de 1 a 2 meses da data da listagem, mas alguns são bastante antigos, com o mais antigo em torno de 9 meses.

In [39]:
autos["registration_year"].describe()

count    360635.000000
mean       2004.433133
std          81.016977
min        1000.000000
25%        1999.000000
50%        2004.000000
75%        2008.000000
max        9999.000000
Name: registration_year, dtype: float64

O ano em que o carro foi registrado pela primeira vez provavelmente indicará a idade do carro. Olhando para esta coluna, notamos alguns valores ímpares. O valor mínimo é 1000, muito antes dos carros serem inventados e o máximo é 9999, muitos anos no futuro.

## Lidando com dados de ano de registro incorretos

Como um carro não pode ser registrado primeiro antes que a listagem seja vista, qualquer veículo com ano de registro acima de 2016 é definitivamente impreciso. Determinar o primeiro ano válido é mais difícil. Realisticamente, poderia estar em algum lugar nas primeiras décadas de 1900.

Uma opção é remover as listagens com esses valores. Vamos determinar qual porcentagem de nossos dados tem valores inválidos nesta coluna:

In [40]:
(~autos["registration_year"].between(1900,2016)).sum() / autos.shape[0]

0.038751091824143526

Dado que isso é menos de 4% de nossos dados, removeremos essas linhas.

In [41]:
# Many ways to select rows in a dataframe that fall within a value range for a column.
# Using `Series.between()` is one way.
autos = autos[autos["registration_year"].between(1900,2016)]
autos["registration_year"].value_counts(normalize=True).head(10)

2000    0.066699
1999    0.063552
2005    0.062669
2006    0.057708
2001    0.056955
2003    0.056557
2004    0.056173
2002    0.054290
2007    0.050499
1998    0.049691
Name: registration_year, dtype: float64

Parece que a maioria dos veículos foi registrada pela primeira vez nos últimos 20 anos.

## Explorando o preço por marca

In [42]:
autos["brand"].value_counts(normalize=True)

volkswagen        0.211700
bmw               0.109871
opel              0.106410
mercedes_benz     0.096841
audi              0.089543
ford              0.068918
renault           0.047516
peugeot           0.030153
fiat              0.025691
seat              0.018661
skoda             0.015687
mazda             0.015384
smart             0.014331
citroen           0.013950
nissan            0.013598
toyota            0.012932
hyundai           0.009972
sonstige_autos    0.009493
mini              0.009384
volvo             0.009147
mitsubishi        0.008236
honda             0.007532
kia               0.006915
suzuki            0.006364
alfa_romeo        0.006309
porsche           0.006211
chevrolet         0.005022
chrysler          0.003863
dacia             0.002495
jeep              0.002192
land_rover        0.002166
daihatsu          0.002161
subaru            0.002117
jaguar            0.001734
saab              0.001465
daewoo            0.001457
trabant           0.001408
l

Os fabricantes alemães representam quatro das cinco principais marcas, quase 50% das listas gerais. A Volkswagen é de longe a marca mais popular, com aproximadamente o dobro dos carros à venda das próximas duas marcas combinadas.

Existem muitas marcas que não possuem uma porcentagem significativa de listagens, portanto, limitaremos nossa análise às marcas que representam mais de 5% do total de listagens.

In [43]:
brand_counts = autos["brand"].value_counts(normalize=True)
common_brands = brand_counts[brand_counts > .05].index
print(common_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


In [44]:
brand_mean_prices = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_price = brand_only["price"].mean()
    brand_mean_prices[brand] = int(mean_price)

brand_mean_prices

{'volkswagen': 5400,
 'bmw': 8449,
 'opel': 2971,
 'mercedes_benz': 8551,
 'audi': 9086,
 'ford': 3696}

Das 5 principais marcas, há uma diferença de preço distinta:

Audi, BMW e Mercedes Benz são mais caros

Ford e Opel são menos caros

A Volkswagen está no meio - isso pode explicar sua popularidade, pode ser a opção "melhor dos dois mundos".

## Explorando a milhagem

In [45]:
bmp_series = pd.Series(brand_mean_prices)
pd.DataFrame(bmp_series, columns=["mean_price"])

,mean_price
volkswagen,5400
bmw,8449
opel,2971
mercedes_benz,8551
audi,9086
ford,3696


In [46]:
brand_mean_mileage = {}

for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_mileage = brand_only["odometer_km"].mean()
    brand_mean_mileage[brand] = int(mean_mileage)

mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending=False)
mean_prices = pd.Series(brand_mean_prices).sort_values(ascending=False)

In [47]:
brand_info = pd.DataFrame(mean_mileage,columns=['mean_mileage'])
brand_info

,mean_mileage
bmw,132800
mercedes_benz,130572
audi,129443
opel,128722
volkswagen,128386
ford,123662


In [48]:
brand_info["mean_price"] = mean_prices
brand_info

,mean_mileage,mean_price
bmw,132800,8449
mercedes_benz,130572,8551
audi,129443,9086
opel,128722,2971
volkswagen,128386,5400
ford,123662,3696


A gama de quilometragem dos carros não varia tanto quanto os preços por marca, em vez disso, todos caem dentro de 10% para as marcas principais. Existe uma ligeira tendência para os veículos mais caros terem maior quilometragem, com os veículos mais baratos a terem uma quilometragem mais baixa.